In [ ]:
!!pip install "snowflake-connector-python[pandas]"

In [ ]:
!pip install -q matplotlib seaborn xgboost

In [1]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting

import seaborn as sns # for making plots with seaborn
color = sns.color_palette()

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import datetime

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

In [2]:
#Import all snowflake connection details from template variables.

db_user = os.getenv('sf_user')
db_password =  'Password@2023' #os.getenv('sf_password')
db_account = os.getenv('sf_account')
db_database =  os.getenv('sf_db')
db_role = os.getenv('sf_role')
db_warehouse = 'FOSFOR_FDC'
db_schema = 'PUBLIC'

In [10]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [15]:
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

cc_customer = session.table('FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA').to_pandas()
cc_payment = session.table('FDC_DATA_MANISH.PUBLIC.CC_PAYMENT_DATA').to_pandas()

In [16]:
# Glimpse of Data
print('Size of Custommer data', cc_customer.shape)
print('Size of Payment details', cc_payment.shape)

Size of Custommer data (1125, 13)
Size of Payment details (8250, 12)


In [17]:
cc_customer.head()

,LABEL,ID,FEA_1,FEA_2,FEA_3,FEA_4,FEA_5,FEA_6,FEA_7,FEA_8,FEA_9,FEA_10,FEA_11
0,1.0,54982665.0,5.0,1245.5,3.0,77000.0,2.0,15.0,5.0,109.0,5.0,151300.0,244.948974
1,0.0,54989781.0,4.0,1415.0,3.0,93000.0,2.0,8.0,5.0,113.0,4.0,151300.0,273.861279
2,0.0,58998054.0,4.0,1250.0,3.0,137000.0,2.0,8.0,5.0,90.0,5.0,72000.0,1.000000
3,0.0,58995381.0,7.0,1220.0,3.0,76000.0,2.0,11.0,2.0,90.0,5.0,71002.0,1.000000
4,0.0,58987926.0,5.0,0.0,2.0,50000.0,2.0,15.0,5.0,108.0,4.0,450000.0,169.000000


In [18]:
cc_payment.head()

,ID,OVD_T1,OVD_T2,OVD_T3,OVD_SUM,PAY_NORMAL,PROD_CODE,PROD_LIMIT,UPDATE_DATE,NEW_BALANCE,HIGHEST_BALANCE,REPORT_DATE
0,58987402.0,0.0,0.0,0.0,0.0,1.0,10.0,16500.0,16500.0,0.0,0.0,0.0
1,58995151.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,588720.0,491100.0,491100.0
2,58997200.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,840000.0,700500.0,700500.0
3,54988608.0,0.0,0.0,0.0,0.0,3.0,10.0,37400.0,37400.0,8425.2,7520.0,7520.0
4,54987763.0,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,15147.6,0.0,0.0
